In [11]:
# Step 1: Librerie necessarie
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
# # Step 2: Funzione per calcolo sentiment
# def calculate_sentiment(text):
#     analyzer = SentimentIntensityAnalyzer()
#     scores = analyzer.polarity_scores(text)
#     return scores

In [13]:
# sentences = [
#     # "VADER is smart, handsome, and funny.",  # positive sentence example
#     # "VADER is not smart, handsome, nor funny.",  # negation sentence example
#     # "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
#     # "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
#     # "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
#     # "VADER is VERY SMART, handsome, and FUNNY!!!",  # combination of signals - VADER appropriately adjusts intensity
#     # "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!",  # booster words & punctuation make this close to ceiling for score
#     # "The book was good.",  # positive sentence
#     # "The book was kind of good.",  # qualified positive sentence is handled correctly (intensity adjusted)
#     # "The plot was good, but the characters are uncompelling and the dialog is not great.",  # mixed negation sentence
#     # "At least it isn't a horrible book.",  # negated negative sentence with contraction
#     # "Make sure you :) or :D today!",  # emoticons handled
#     # "Today SUX!",  # negative slang with capitalization emphasis
#     # "Today only kinda sux! But I'll get by, lol",  # mixed sentiment example with slang and constrastive conjunction "but"
#     # "The weather is lousy today",  # mixed sentiment with contrastive conjunction
#     # "The family adopted those puppies. ",
#     "Last break, I completed all of my homework ahead of time. "
#     "Last night I did a lot of homework. ",
#     "know that I should enter the haunted house at my own risk.",
#     "I love action movies, not boring dramas.",
#     "I watched a movie last night and the actor was good.",
#     "My teacher is a good advisior.",
#     "Alcohol is not always good.",
#     "I tend to exercise after class.",
#     "Last night, there was an accident and traffic was horrible.",
#     "I watched a movie last night. I was really scared.",
#     "My mom is getting old.",
#     "Age isn't important.",
#     "My ancestors are from Germany.",
#     "I watched a movie last night and the actor was good.",
#     "I was picked to join the club. Wow, it was an honor.",
#     "I love riding in airplanes and flying different places.",
#     "My ancestors are from Germany.",
#     "Last night, there was an accident and traffic was horrible.",
#     "My boss works in an office right above me.",
#     "I have an appointment this afternoon with my teacher.",
#     "I asked my teacher for some advice and we had a discussion.",
#     "I noticed my sister acting strange.",
#     "Last night I did a lot of homework.",
#     "My boss works in an office right above me.",
#     "I got a D in the class; I accepted it.",
#     "I watched a movie last night and the actor was good.",
#     "I noticed my sister acting strange.",
#     "I love action movies, not boring dramas.",
#     "I asked my teacher for some advice and we had a discussion.",
#     "My teacher is a good advisior.",
#     "I watched a movie last night. I was really scared.",
#     "This afternoon I'm going to the store.",
# ]

In [14]:
# # Step 3: Estrarre punteggi di sentiment
# analyzer = SentimentIntensityAnalyzer()
# for sentence in sentences:
#     vs = analyzer.polarity_scores(sentence)
#     print("{:-<65} {}".format(sentence, str(vs)))

In [15]:
# Script per leggere il file Excel e creare un mapping tra nomi video e captions
import pandas as pd
import os

# Percorso del file Excel
excel_path = "../data/raw/train/how2sign_train.xlsx"
video_folder = "../data/raw/train/raw_videos_front_train"

# Legge il file Excel
print("Leggendo il file Excel...")
df = pd.read_excel(excel_path)

# Mostra le prime righe per capire la struttura
# print("Prime 5 righe del dataset:")
# print(df.head())
print(f"\nColonne disponibili: {list(df.columns)}")
print(f"Numero totale di righe: {len(df)}")

# Estrae le frasi (assumendo che sia l'ultima colonna)
sentences = df.iloc[:, -1].dropna().tolist()

# Assumendo che ci sia una colonna con i nomi dei video
if "SENTENCE_NAME" in df.columns:
    video_names = df["SENTENCE_NAME"].dropna().tolist()

# Filtra i video che esistono nella cartella raw_videos
existing_videos = set(os.listdir(video_folder))
video_caption_mapping = {}
for i, (video, caption) in enumerate(zip(video_names, sentences)):
    video_file = f"{video}.mp4"
    if video_file in existing_videos and pd.notna(caption):
        video_caption_mapping[video] = caption

print(
    f"\nCreato mapping per {len(video_caption_mapping)} video-caption pairs (solo video esistenti)"
)
print("Esempi di mapping:")
for i, (video, caption) in enumerate(list(video_caption_mapping.items())[:3]):
    print(f"{i+1}. Video: {video}")
    print(f"   Caption: {caption[:100]}...")

Leggendo il file Excel...

Colonne disponibili: ['VIDEO_ID', 'VIDEO_NAME', 'SENTENCE_ID', 'SENTENCE_NAME', 'START', 'END', 'SENTENCE']
Numero totale di righe: 31165

Creato mapping per 2147 video-caption pairs (solo video esistenti)
Esempi di mapping:
1. Video: --7E2sU6zP4_10-5-rgb_front
   Caption: And I call them decorative elements because basically all they're meant to do is to enrich and color...
2. Video: --7E2sU6zP4_11-5-rgb_front
   Caption: So they don't really have much of a symbolic meaning other than maybe life is richer, life is beauti...
3. Video: --7E2sU6zP4_12-5-rgb_front
   Caption: Now this is very, this is actually an insert of a kind of an envelope for stationary, and this is a ...


In [16]:
# Step 4: Calcola sentiment per ogni video-caption pair e organizza per categorie
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use("Agg")
from collections import Counter

analyzer = SentimentIntensityAnalyzer()

threshold = 0.65  # Soglia per classificare come positivo o negativo

# Calcola il sentiment per ogni caption
video_sentiment_data = []
for video_name, caption in video_caption_mapping.items():
    scores = analyzer.polarity_scores(caption)
    compound_score = scores["compound"]

    # Determina la categoria
    if compound_score >= threshold:
        emotion = "Positive"
    elif compound_score <= -threshold:
        emotion = "Negative"
    else:
        emotion = "Neutral"

    video_sentiment_data.append(
        {
            "video_name": video_name,
            "caption": caption,
            "compound_score": compound_score,
            "emotion": emotion,
            "scores": scores,
        }
    )

# Organizza per categoria
positive_examples = [
    item for item in video_sentiment_data if item["emotion"] == "Positive"
]
negative_examples = [
    item for item in video_sentiment_data if item["emotion"] == "Negative"
]
neutral_examples = [
    item for item in video_sentiment_data if item["emotion"] == "Neutral"
]

# Conta le occorrenze
emotion_counts = Counter([item["emotion"] for item in video_sentiment_data])

print(f"Distribuzione sentiment:")
print(f"Positive: {len(positive_examples)} esempi")
print(f"Negative: {len(negative_examples)} esempi")
print(f"Neutral: {len(neutral_examples)} esempi")
print(f"Totale: {len(video_sentiment_data)} esempi")

# Mostra esempi per ogni categoria
print("\nEsempi positivi:")
for example in positive_examples[:3]:
    print(f"Video: {example['video_name']}, Caption: {example['caption'][:200]}+++")

print("\nEsempi negativi:")
for example in negative_examples[:3]:
    print(f"Video: {example['video_name']}, Caption: {example['caption'][:200]}+++")

print("\nEsempi neutri:")
for example in neutral_examples[:3]:
    print(f"Video: {example['video_name']}, Caption: {example['caption'][:200]}+++")

# Crea il grafico
plt.figure(figsize=(10, 6))
bars = plt.bar(
    emotion_counts.keys(), emotion_counts.values(), color=["green", "red", "gray"]
)
plt.title("Distribuzione delle emozioni")
plt.xlabel("Emozione")
plt.ylabel("Numero di video")

# Aggiungi il numero in cima a ogni colonna
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{int(height)}",
        ha="center",
        va="bottom",
        fontsize=10,
    )

# Aggiungi la soglia usata come testo in alto a destra
plt.text(
    0.95,
    0.95,
    f"Threshold: |{threshold}|",
    transform=plt.gca().transAxes,
    fontsize=10,
    verticalalignment="top",
    horizontalalignment="right",
    bbox=dict(facecolor="white", alpha=0.5),
)

# Salva l'immagine nella cartella specificata
output_folder = "../reports/figures"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, f"emotions_distribution_{threshold}.png")
plt.savefig(output_path)

Distribuzione sentiment:
Positive: 213 esempi
Negative: 28 esempi
Neutral: 1906 esempi
Totale: 2147 esempi

Esempi positivi:
Video: --7E2sU6zP4_11-5-rgb_front, Caption: So they don't really have much of a symbolic meaning other than maybe life is richer, life is beautiful, but they've become so beautifully stylized and so you find them in different illuminative being+++
Video: --8pSDeC-fg_14-5-rgb_front, Caption: By doing cognitive therapy and working on those thoughts, we can help our clients to feel better about themselves by gaining a more accurate appraisal of themselves, their situation and their environm+++
Video: -00cp1iGiDw_10_11-5-rgb_front, Caption: Now make sure your client is comfortable in the side position, supporting your head, knee supported, arms are resting in front comfortably.+++

Esempi negativi:
Video: -06_nJnhORg_18-5-rgb_front, Caption: It works right away and it will actually kill all of the fleas that are on the dog for the next 24 hours and it start working i

In [17]:
# Estrai tutti i valori 'compound'
compound_scores = [res["compound_score"] for res in video_sentiment_data]

# Crea l'istogramma
plt.hist(compound_scores, bins=10, color="skyblue", edgecolor="black")
plt.title("Distribuzione dei punteggi 'compound' (sentiment)")
plt.xlabel("Valore compound")
plt.ylabel("Numero di frasi")
plt.show()

/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_31590/290048260.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [18]:
# Salva i video positivi e negativi con le relative frasi in un file CSV
import csv

# Percorso del file CSV di output
output_csv_path = f"../data/processed/train/video_sentiment_data_{threshold}.csv"

# Filtra i dati positivi e negativi
positive_negative_data = [
    {
        "video_name": item["video_name"],
        "caption": item["caption"],
        "emotion": item["emotion"],
    }
    for item in video_sentiment_data
    if item["emotion"] in ["Positive", "Negative"]
]

# Scrive i dati in un file CSV
with open(output_csv_path, mode="w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ["video_name", "caption", "emotion"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(positive_negative_data)

print(f"File CSV salvato con successo in: {output_csv_path}")

File CSV salvato con successo in: ../data/processed/train/video_sentiment_data_0.65.csv
